#

In [1]:
import torch
import numpy as np
import sys
sys.path.append('../')
from models import Model
import datasets
from tqdm.notebook import tqdm
from utils.hyp_cone_utils import cone_distance_sum
from utils.poincare_distance import poincare_distance
from losses import compute_mask

In [2]:
def compute_mask(args, sizes, B):
    if args.use_labels:
        return None, None  # No need to compute mask

    last_size, size_gt, size_pred = sizes[0]

    # mask meaning: -2: omit, -1: temporal neg (hard), 0: easy neg, 1: pos, -3: spatial neg
    mask = torch.zeros((B, size_pred, last_size ** 2, B, size_gt, last_size ** 2), dtype=torch.int8, requires_grad=False).detach().cuda()

    mask[torch.arange(B), :, :, torch.arange(B), :, :] = -3  # spatial neg

    if args.early_action_self:
        pass  # Here NO temporal neg! All steps try to predict the last one
    else:
        for k in range(B):
            mask[k, :, torch.arange(last_size ** 2), k, :, torch.arange(last_size ** 2)] = -1  # temporal neg

    tmp = mask.permute(0,2,1,3,5,4).reshape(B * last_size ** 2, size_pred, B * last_size ** 2, size_gt)

    if args.early_action_self:
        tmp[torch.arange(B * last_size ** 2), :, torch.arange(B * last_size ** 2)] = 1  # pos
    else:
        assert size_gt == size_pred
        for j in range(B * last_size ** 2):
            tmp[j, torch.arange(size_pred), j, torch.arange(size_gt)] = 1  # pos

    mask = tmp.view(B, last_size ** 2, size_pred, B, last_size ** 2, size_gt).permute(0, 2, 1, 3, 5, 4)

    # Now, given task mask as input, compute the target for contrastive loss
    if mask is None:
        return None, None
    # dot product is computed in parallel gpus, so get less easy neg, bounded by batch size in each gpu'''
    # mask meaning: -2: omit, -1: temporal neg (hard), 0: easy neg, 1: pos, -3: spatial neg
    (B, NP, SQ, B2, NS, _) = mask.size()  # [B, P, SQ, B, N, SQ]
    target = mask == 1
    target.requires_grad = False
    return target, (B, B2, NS, NP, SQ)

In [3]:
import os
device_ids = [0, 1]
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
model_path = '/proj/vondrick/didac/code/DPC/logs/log_train_dpc_hyper_v1_poincare_kinetics/20201019_195227/model/model_best_epoch159.pth.tar'

In [5]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [6]:
args = Namespace(hyperbolic=True, \
                 hyperbolic_version=1, \
                 network_feature='resnet18', \
                 distance='squared', \
                 early_action=False, \
                 early_action_self=False, \
                 dataset='k600', \
                 pred_step=3, \
                 seq_len=5, \
                 num_seq=8, \
                 ds=3, \
                 img_dim=128, \
                 batch_size=32, \
                 fp16=True, \
                 fp64_hyper=True, \
                 use_labels=False, \
                 n_classes=0, \
                 linear_input=True, \
                 hierarchical_labels=False, \
                 action_level_gt=False, \
                 num_workers=16, \
                 cross_gpu_score=True
                 )

In [7]:
model = Model(args).cuda()

In [8]:
checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['state_dict'], strict=True)
model = torch.nn.DataParallel(model, device_ids=device_ids)
model.eval()

DataParallel(
  (module): Model(
    (backbone): ResNet2d3d_full(
      (conv1): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock2d(
          (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (relu): ReLU(inplace=True)
          (conv2): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
        (1): BasicBlock2d(
          (conv1): Conv3d(64, 64, kernel_size=(1, 3, 

In [9]:
from torchvision import transforms
from utils import augmentation
from datasets import Kinetics600_full_3d
from torch.utils import data
transform = transforms.Compose([
            augmentation.RandomSizedCrop(size=args.img_dim, consistent=True, p=1.0),
            augmentation.RandomHorizontalFlip(consistent=True),
            augmentation.RandomGray(consistent=False, p=0.5),
            augmentation.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.25, p=1.0),
            augmentation.ToTensor(),
            augmentation.Normalize()
        ])
dataset = Kinetics600_full_3d(mode='test',
                              transform=transform,
                              seq_len=args.seq_len,
                              num_seq=args.num_seq,
                              downsample=5,
                              return_label=False,
                              vis=True)
dataloader = data.DataLoader(dataset,
                             batch_size=args.batch_size,
                             shuffle=False,
                             num_workers=args.num_workers,
                             pin_memory=True,
                             drop_last=True)

# test model loading and dataloading

In [21]:
idx, (input_dict, label) = next(enumerate(dataloader))

In [26]:
len(input_dict['vpath']), input_dict['idx_block'].shape

(32, torch.Size([32, 40]))

In [ ]:
input_seq = input_seq.cuda()

In [ ]:
pred, feature_dist, sizes = model(input_seq)

# spatial pooling embedding

In [ ]:
target, (B, B2, NS, NP, SQ) = compute_mask(args, sizes, args.batch_size)
_, D = pred.shape
pred = pred.reshape(B, NP, SQ, D)
feature_dist = feature_dist.reshape(B, NS, SQ, D)
pred_pooled = torch.mean(pred, dim=2).reshape(-1, D)
feature_dist_pooled = torch.mean(feature_dist, dim=2).reshape(-1, D)

In [ ]:
pred_pooled = pred_pooled.reshape(B, NP, D)
feature_dist_pooled = feature_dist_pooled.reshape(B, NS, D)

In [ ]:
# poincare_distance(pred_pooled, feature_dist_pooled)

# collect all feature_dist from Kinetics

In [ ]:
torch.cuda.empty_cache()

In [28]:
all_features = []
all_preds = []
all_vpaths = []
all_idx_blocks = []

In [29]:
with torch.no_grad():
    for idx, (input_dict, label) in tqdm(enumerate(dataloader), total=len(dataloader)):
        input_seq = input_dict['t_seq'].cuda()
        pred, feature_dist, sizes = model(input_seq)
        target, (B, B2, NS, NP, SQ) = compute_mask(args, sizes, args.batch_size)
        _, D = pred.shape
        pred = pred.reshape(B, NP, SQ, D)
        feature_dist = feature_dist.reshape(B, NS, SQ, D)
        pred_pooled = torch.mean(pred, dim=2).reshape(-1, D)
        feature_dist_pooled = torch.mean(feature_dist, dim=2).reshape(-1, D)
        del pred, feature_dist
        pred_pooled = pred_pooled.reshape(B, NP, D)
        feature_dist_pooled = feature_dist_pooled.reshape(B, NS, D)
        all_features.append(feature_dist_pooled.cpu().detach())
        all_preds.append(pred_pooled.cpu().detach())
        all_vpaths.extend(input_dict['vpath'])
        all_idx_blocks.append(input_dict['idx_block'])

../backbone/hyrnn_nets.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  k = torch.tensor(k)
Traceback (most recent call last):
  File "/proj/vondrick/ruoshi/anaconda3/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/proj/vondrick/ruoshi/anaconda3/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/proj/vondrick/ruoshi/anaconda3/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


Error with image in path /local/vondrick/ruoshi/k600/extracted_frames/train/arguing/25fps5EddqmM0QJY_000032_000042/image_00199.jpg
Error with image in path /local/vondrick/ruoshi/k600/extracted_frames/train/arguing/25fps4O-6WE8flyg_000025_000035/image_00091.jpg
Error with image in path /local/vondrick/ruoshi/k600/extracted_frames/train/arguing/25fpsOGYgdjNeoD8_000020_000030/image_00062.jpg
Error with image in path /local/vondrick/ruoshi/k600/extracted_frames/train/archery/25fps1Y3VfZz-sYc_000029_000039/image_00041.jpg



KeyboardInterrupt: 

In [30]:
all_features = torch.cat(all_features)
all_preds = torch.cat(all_preds)
all_idx_blocks = torch.cat(all_idx_blocks)

In [32]:
all_idx_blocks.shape, all_features.shape, all_preds.shape

(torch.Size([5600, 40]),
 torch.Size([5600, 3, 256]),
 torch.Size([5600, 3, 256]))

In [47]:
features_info = {'feature': all_features, 'pred': all_preds, 'vpath': all_vpaths, 'idx_block': all_idx_blocks}

In [58]:
import pickle
# model_path = args.pretrain
base_path = '/'.join(model_path.split('/')[:-2])
embedding_path = os.path.join(base_path, 'embeds')
if not os.path.exists(embedding_path):
    os.makedirs(embedding_path)

f = open(os.path.join(embedding_path, model_path.split('/')[-1][:-8] + '_embeds.pkl'),'wb')
pickle.dump(features_info,f)
f.close()

In [46]:
'/'.join(model_path.split('/')[:-2])

'/proj/vondrick/didac/code/DPC/logs/log_train_dpc_hyper_v1_poincare_kinetics/20201019_195227'

In [56]:
model_path.split('/')[-1][:-8] + '_embeds.pkl'

'model_best_epoch159embeds'